In [1]:
import dgl
import time
import torch as th
import numpy as np
import torch.nn.functional as F

from load_data import load_ogb
from model import SAGE, GAT, get_eigen
from train_eval import train_ft_embed

Using backend: pytorch


This notebook shows the performance of GAT on the OGBN-products graph with trainable embeddings on the nodes. We use scaled eigenvectors to initialize the node embeddings. The logic here is that if eigenvectors can serve as positional node embeddings, fine-tuning the node embeddings should give us even better performance. In this experiment, we tune the node embeddings with the same learning rate as the one for the GraphSage model.

In [2]:
g, num_labels = load_ogb('ogbn-products')
g.ndata.pop('features')

in_feats = 128
eigen_vals, eigen_vecs = get_eigen(g, in_feats, 'ogbn-products')
nfeat = th.tensor(eigen_vecs * np.sqrt(eigen_vals).reshape((1, len(eigen_vals))), dtype=th.float32)
labels = g.ndata.pop('labels')

hyperparams = {
    'batch_size': 512,
    'num_workers': 0,
    'num_hidden': 256,
    'num_layers': 3,
    'num_heads': 2,
    'dropout': 0.5,
    'lr': 0.003,
    'sparse_lr': 0.003,
    'num_epochs': 20,
    'fanouts': [15,10,5],
    'eval_batch_size': 10000
}
device = th.device('cuda:4')

# Pack data
data = g, nfeat, labels
# Define model
model = GAT(in_feats, hyperparams['num_hidden'], num_labels,
             hyperparams['num_layers'], hyperparams['num_heads'], F.relu, hyperparams['dropout'])
train_ft_embed(model, data, hyperparams, device, 1)

load ogbn-products
finish loading ogbn-products
finish constructing ogbn-products


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  


embedding is in  cuda:4


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 3.8503 | Train Acc 0.0117 | Speed (samples/sec) nan | GPU 4495.0 MB
Epoch 00000 | Step 00020 | Loss 1.4978 | Train Acc 0.6328 | Speed (samples/sec) 278.2007 | GPU 6369.6 MB
Epoch 00000 | Step 00040 | Loss 0.7826 | Train Acc 0.8145 | Speed (samples/sec) 296.4423 | GPU 6369.6 MB
Epoch 00000 | Step 00060 | Loss 0.5175 | Train Acc 0.8594 | Speed (samples/sec) 299.4021 | GPU 6388.3 MB
Epoch 00000 | Step 00080 | Loss 0.4420 | Train Acc 0.8809 | Speed (samples/sec) 298.9589 | GPU 6389.3 MB
Epoch 00000 | Step 00100 | Loss 0.3912 | Train Acc 0.8906 | Speed (samples/sec) 297.5999 | GPU 6389.3 MB
Epoch 00000 | Step 00120 | Loss 0.4164 | Train Acc 0.8926 | Speed (samples/sec) 295.1031 | GPU 6389.6 MB
Epoch 00000 | Step 00140 | Loss 0.3759 | Train Acc 0.8984 | Speed (samples/sec) 292.9337 | GPU 6389.6 MB
Epoch 00000 | Step 00160 | Loss 0.4102 | Train Acc 0.9023 | Speed (samples/sec) 291.5703 | GPU 6389.6 MB
Epoch 00000 | Step 00180 | Loss 0.3854 | Train Acc 0.8945 | 

...

Val acc 0.9208, Test Acc: 0.7857
Best val acc 0.9208, best test acc: 0.7857
Epoch 00018 | Step 00000 | Loss 0.2427 | Train Acc 0.9316 | Speed (samples/sec) 322.7315 | GPU 6412.1 MB
Epoch 00018 | Step 00020 | Loss 0.2687 | Train Acc 0.9277 | Speed (samples/sec) 323.0686 | GPU 6418.3 MB
Epoch 00018 | Step 00040 | Loss 0.2224 | Train Acc 0.9316 | Speed (samples/sec) 323.4118 | GPU 6418.3 MB
Epoch 00018 | Step 00060 | Loss 0.2404 | Train Acc 0.9219 | Speed (samples/sec) 323.7349 | GPU 6418.3 MB
Epoch 00018 | Step 00080 | Loss 0.2329 | Train Acc 0.9219 | Speed (samples/sec) 324.0566 | GPU 6418.3 MB
Epoch 00018 | Step 00100 | Loss 0.2267 | Train Acc 0.9316 | Speed (samples/sec) 324.3889 | GPU 6418.3 MB
Epoch 00018 | Step 00120 | Loss 0.2249 | Train Acc 0.9316 | Speed (samples/sec) 324.7197 | GPU 6418.3 MB
Epoch 00018 | Step 00140 | Loss 0.2306 | Train Acc 0.9277 | Speed (samples/sec) 325.0130 | GPU 6418.3 MB
Epoch 00018 | Step 00160 | Loss 0.1816 | Train Acc 0.9395 | Speed (samples/sec) 325.

100%|██████████| 245/245 [01:24<00:00,  2.88it/s]


Val acc 0.9214, Test Acc: 0.7918
Best val acc 0.9214, best test acc: 0.7918
Epoch 00019 | Step 00000 | Loss 0.2737 | Train Acc 0.9043 | Speed (samples/sec) 328.7877 | GPU 6418.3 MB
Epoch 00019 | Step 00020 | Loss 0.2635 | Train Acc 0.9082 | Speed (samples/sec) 329.0830 | GPU 6418.3 MB
Epoch 00019 | Step 00040 | Loss 0.2917 | Train Acc 0.9121 | Speed (samples/sec) 329.3846 | GPU 6418.3 MB
Epoch 00019 | Step 00060 | Loss 0.2869 | Train Acc 0.8945 | Speed (samples/sec) 329.6826 | GPU 6418.3 MB
Epoch 00019 | Step 00080 | Loss 0.2784 | Train Acc 0.9180 | Speed (samples/sec) 329.9836 | GPU 6418.3 MB
Epoch 00019 | Step 00100 | Loss 0.2326 | Train Acc 0.9258 | Speed (samples/sec) 330.2733 | GPU 6418.3 MB
Epoch 00019 | Step 00120 | Loss 0.2387 | Train Acc 0.9180 | Speed (samples/sec) 330.5601 | GPU 6418.3 MB
Epoch 00019 | Step 00140 | Loss 0.2430 | Train Acc 0.9219 | Speed (samples/sec) 330.8626 | GPU 6418.3 MB
Epoch 00019 | Step 00160 | Loss 0.2242 | Train Acc 0.9199 | Speed (samples/sec) 331.

100%|██████████| 245/245 [01:25<00:00,  2.87it/s]


Val acc 0.9176, Test Acc: 0.7888
Avg epoch time: 766.3484711488088
Best val acc 0.9214, best test acc: 0.7918
